In [2]:
#!/usr/bin/env python36
# -*- coding: utf-8 -*-

import argparse
import time
import csv
import pickle
import operator
import datetime
import os
class datatype:
    def __init__(self, dataset):
        self.dataset = dataset


opt = datatype('diginetica')
# opt.dataset = 'diginetica'
print(opt.dataset)
# opt.dataset = 'retailrocket'
if opt.dataset == 'diginetica':
    dataset = './data/diginetica/train-item-views.csv'
elif opt.dataset == 'retailrocket':
    dataset = './data/retailrocket/events_train_full.0.txt'

print("-- Starting @ %ss" % datetime.datetime.now())
with open(dataset, "r") as f:
    if opt.dataset == 'retailrocket':
        reader = csv.DictReader(f, delimiter='\t', fieldnames=['timestamp', 'user_id', 'item_id', 'session_id'])
    else:
        reader = csv.DictReader(f, delimiter=';', fieldnames=['session_id', 'user_id', 'item_id', 'timestamp', 'eventdate'])
        next(reader)
    sess_clicks = {}
    sess_date = {}
    ctr = 0
    curid = -1
    curdate = None
    isfirstline = 1
    for data in reader:
        if isfirstline and opt.dataset == 'retailrocket':
            isfirstline = 0
            continue
        sessid = data['session_id']
        if curdate and not curid == sessid:
            date = ''
            if opt.dataset == 'retailrocket':
                date = int(curdate)
            else:
                date = time.mktime(time.strptime(curdate, '%Y-%m-%d'))
            sess_date[curid] = date
        curid = sessid
        if opt.dataset == 'retailrocket':
            item = data['item_id']
        else:
            item = data['item_id']
        curdate = ''
        if  opt.dataset == 'retailrocket':
            curdate = data['timestamp']
        else:
            curdate = data['eventdate']

        if sessid in sess_clicks:
            sess_clicks[sessid] += [item]
        else:
            sess_clicks[sessid] = [item]
        ctr += 1
    date = ''
    if opt.dataset == 'retailrocket':
        date = int(curdate)
    else:
        date = time.mktime(time.strptime(curdate, '%Y-%m-%d'))
    sess_date[curid] = date
print("-- Reading data @ %ss" % datetime.datetime.now())

# Filter out length 1 sessions
for s in list(sess_clicks):
    if len(sess_clicks[s]) <= 2:
        del sess_clicks[s]
        del sess_date[s]

# Count number of times each item appears
iid_counts = {}
for s in sess_clicks:
    seq = sess_clicks[s]
    for iid in seq:
        if iid in iid_counts:
            iid_counts[iid] += 1
        else:
            iid_counts[iid] = 1

sorted_counts = sorted(iid_counts.items(), key=operator.itemgetter(1))
length = len(sess_clicks)
max_len = 3
for s in list(sess_clicks):
    curseq = sess_clicks[s]
    filseq = list(filter(lambda i: iid_counts[i] >= 5, curseq))
    if len(filseq) < max_len:
        del sess_clicks[s]
        del sess_date[s]
    else:
        sess_clicks[s] = filseq

# Split out test set based on dates
dates = list(sess_date.items())
maxdate = dates[0][1]

for _, date in dates:
    if maxdate < date:
        maxdate = date

# 7 days for test
splitdate = maxdate - 86400 * 7

print('Splitting date', splitdate)      
tra_sess = filter(lambda x: x[1] < splitdate, dates)
tes_sess = filter(lambda x: x[1] > splitdate, dates)

# Sort sessions by date
tra_sess = sorted(tra_sess, key=operator.itemgetter(1))
tes_sess = sorted(tes_sess, key=operator.itemgetter(1))
print("Train Session Length:", len(tra_sess))
print("Test Session Length:", len(tes_sess))
print(tra_sess[:3])
print(tes_sess[:3])
print("-- Splitting train set and test set @ %ss" % datetime.datetime.now())

# Choosing item count >=5 gives approximately the same number of items as reported in paper
item_dict = {}
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_ids = []
    train_seqs = []
    train_dates = []
    item_ctr = 1
    for s, date in tra_sess:
        seq = sess_clicks[s]
        outseq = []
        if len(seq) < max_len:
            continue
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1

        train_ids += [s]
        train_dates += [date]
        train_seqs += [outseq]
    print('obtain_tra-------item_num=', item_ctr)
    return train_ids, train_dates, train_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_ids = []
    test_seqs = []
    test_dates = []
    for s, date in tes_sess:
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < max_len:
            continue
        test_ids += [s]
        test_dates += [date]
        test_seqs += [outseq]
    return test_ids, test_dates, test_seqs


tra_ids, tra_dates, tra_seqs = obtian_tra()
tes_ids, tes_dates, tes_seqs = obtian_tes()

def process_seqs(tra_ids, iseqs, idates):
    out_seqs = []
    out_dates = []
    labs = []
    ids = []
    for id, seq, date in zip(tra_ids, iseqs, idates):
        for i in range(1, len(seq) - 1):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
            out_dates += [int(date)]
            ids += [int(id)]
    return out_seqs, out_dates, labs, ids


tr_seqs, tr_dates, tr_labs, tr_ids = process_seqs(tra_ids, tra_seqs, tra_dates)
te_seqs, te_dates, te_labs, te_ids = process_seqs(tes_ids, tes_seqs, tes_dates)
tra = (tr_ids, tr_seqs, tr_labs, tr_dates)
tes = (te_ids, te_seqs, te_labs, te_dates)
all_train_seq = (tra_ids, tra_seqs)


print('training sessions = ', len(tr_seqs))
print('testing sessions = ', len(te_seqs))
print(tr_seqs[:3], tr_dates[:3], tr_labs[:3])
print(te_seqs[:3], te_dates[:3], te_labs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('all clicks: ', all)
print('avg length: ', all / (len(tra_seqs) + len(tes_seqs) * 1.0))
if opt.dataset == 'diginetica':
    pickle.dump(tra, open('./data/diginetica/train.txt', 'wb'))
    pickle.dump(tes, open('./data/diginetica/test.txt', 'wb'))
    pickle.dump(all_train_seq, open('./data/diginetica/all_train_seq.txt', 'wb'))
elif opt.dataset == 'retailrocket':
    pickle.dump(tra, open('./data/retailrocket/train.txt', 'wb'))
    pickle.dump(tes, open('./data/retailrocket/test.txt', 'wb'))
    pickle.dump(all_train_seq, open('./data/retailrocket/all_train_seq.txt', 'wb'))
else:
    if not os.path.exists('sample'):
        os.makedirs('sample')
    pickle.dump(tra, open('sample/train.txt', 'wb'))
    pickle.dump(tes, open('sample/test.txt', 'wb'))
    pickle.dump(all_train_seq, open('sample/all_train_seq.txt', 'wb'))

print('Done.')


diginetica
-- Starting @ 2020-09-14 14:00:39.076059s
-- Reading data @ 2020-09-14 14:00:45.774557s
Splitting date 1464105600.0
Train Session Length: 132501
Test Session Length: 11371
[('4741', 1451577600.0), ('4747', 1451577600.0), ('4748', 1451577600.0)]
[('289', 1464192000.0), ('302', 1464192000.0), ('1862', 1464192000.0)]
-- Splitting train set and test set @ 2020-09-14 14:00:46.534758s
obtain_tra-------item_num= 40841
training sessions =  526135
testing sessions =  44279
[[1, 2], [4, 5, 6, 6, 7, 8], [4, 5, 6, 6, 7]] [1451577600, 1451577600, 1451577600] [3, 9, 8]
[[19824, 18402, 9756, 19834, 19834], [19824, 18402, 9756, 19834], [19824, 18402, 9756]] [1464192000, 1464192000, 1464192000] [33829, 19834, 19834]
all clicks:  858108
avg length:  5.965421593776721
Done.
